## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-27-03-57-18 +0000,bbc,Two National Guard members in critical conditi...,https://www.bbc.com/news/articles/c3r7qpqddnro...
1,2025-11-27-03-55-10 +0000,nyt,Live Updates: 3 Construction Officials Arreste...,https://www.nytimes.com/live/2025/11/26/world/...
2,2025-11-27-03-43-01 +0000,startribune,2 National Guard members shot in an ambush att...,https://www.startribune.com/two-national-guard...
3,2025-11-27-03-41-46 +0000,bbc,Asahi says 1.5 million customers' data leaked ...,https://www.bbc.com/news/articles/ce86n44178no...
4,2025-11-27-03-22-01 +0000,nypost,Alleged DC National Guardsmen shooter was in A...,https://nypost.com/2025/11/26/us-news/alleged-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
48,trump,38
1,national,24
14,fire,17
333,year,16
16,hong,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
266,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...,126
125,2025-11-26-20-37-11 +0000,nypost,Trump vows ‘animal’ who shot 2 National Guards...,https://nypost.com/2025/11/26/us-news/trump-an...,100
95,2025-11-26-21-59-46 +0000,nypost,Trump requests 500 more National Guard troops ...,https://nypost.com/2025/11/26/us-news/trump-re...,98
43,2025-11-27-00-27-54 +0000,nypost,"Afghan national Rahmanullah Lakanwal, who came...",https://nypost.com/2025/11/26/us-news/afghan-n...,98
151,2025-11-26-19-04-14 +0000,nyt,Trump Administration Will Raise Prices for For...,https://www.nytimes.com/2025/11/25/us/politics...,92


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
266,126,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...
87,75,2025-11-26-22-14-20 +0000,bbc,At least 44 dead and hundreds missing after fi...,https://www.bbc.com/news/articles/cvg165d7ppvo...
13,61,2025-11-27-02-53-50 +0000,nypost,"Hunter, Joe Biden pictured together for first ...",https://nypost.com/2025/11/26/us-news/hunter-j...
216,52,2025-11-26-16-23-23 +0000,cbc,New prosecutor won't pursue charges against Tr...,https://www.cbc.ca/news/world/trump-georgia-el...
43,50,2025-11-27-00-27-54 +0000,nypost,"Afghan national Rahmanullah Lakanwal, who came...",https://nypost.com/2025/11/26/us-news/afghan-n...
190,41,2025-11-26-17-31-50 +0000,nyt,Trump Defends Witkoff’s Tactics in Russia-Ukra...,https://www.nytimes.com/2025/11/26/us/politics...
128,41,2025-11-26-20-33-00 +0000,wsj,The job market continues to sputter even as co...,https://www.wsj.com/economy/central-banking/fe...
272,40,2025-11-26-11-13-21 +0000,nypost,Marty Makary touts FDA efforts to crack down o...,https://nypost.com/2025/11/26/us-news/marty-ma...
52,36,2025-11-27-00-00-43 +0000,bbc,Nine ways the Budget could affect you if you'r...,https://www.bbc.com/news/articles/cn4dxmg7jyjo...
12,34,2025-11-27-02-55-28 +0000,nypost,Utah death row inmate with dementia dies 3 mon...,https://nypost.com/2025/11/26/us-news/utah-dea...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
